In [1]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import torch
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
import cv2
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
import torchvision
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import time
import os
#from __future__ import print_function, division, absolute_import
from collections import OrderedDict
import math
from torch.utils import model_zoo

In [2]:
import pretrainedmodels
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'vgg13', 'vgg16', 'squeezenet1_1', 'vgg19_bn', 'inceptionv3', 'resnet18', 'vgg16_bn', 'resnet101', 'vgg19', 'densenet201', 'vgg11', 'alexnet', 'resnet152', 'densenet169', 'squeezenet1_0', 'vgg13_bn', 'densenet161', 'resnet50', 'vgg11_bn', 'resnet34', 'densenet121', 'nasnetamobile', 'nasnetalarge', 'dpn107', 'dpn92', 'dpn98', 'dpn131', 'dpn68', 'dpn68b', 'xception', 'se_resnext101_32x4d', 'se_resnext50_32x4d', 'se_resnet152', 'senet154', 'se_resnet50', 'se_resnet101', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [3]:
print(pretrainedmodels.pretrained_settings['densenet121'])

{'imagenet': {'input_space': 'RGB', 'std': [0.229, 0.224, 0.225], 'input_size': [3, 224, 224], 'url': 'http://data.lip6.fr/cadene/pretrainedmodels/densenet121-fbdb23505.pth', 'mean': [0.485, 0.456, 0.406], 'input_range': [0, 1], 'num_classes': 1000}}


In [4]:
def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax, cmin, cmax
def crop_resize(img0, size=224, pad=16):
    HEIGHT = 137
    WIDTH = 236
    #crop a box around pixels large than the threshold
    #some images contain line at the sides
    ymin,ymax,xmin,xmax = bbox(img0[5:-5,5:-5] > 80)
    #cropping may cut too much, so we need to add it back
    xmin = xmin - 13 if (xmin > 13) else 0
    ymin = ymin - 10 if (ymin > 10) else 0
    xmax = xmax + 13 if (xmax < WIDTH - 13) else WIDTH
    ymax = ymax + 10 if (ymax < HEIGHT - 10) else HEIGHT
    img = img0[ymin:ymax,xmin:xmax]
    #remove lo intensity pixels as noise
    img[img < 28] = 0
    lx, ly = xmax-xmin,ymax-ymin
    l = max(lx,ly) + pad
    #make sure that the aspect ratio is kept in rescaling
    img = np.pad(img, [((l-ly)//2,), ((l-lx)//2,)], mode='constant')
    return cv2.resize(img,(size,size))

In [6]:
train_transform = transforms.Compose([#transforms.Lambda(crop_resize),
                                      transforms.Lambda(lambda x: crop_resize(x, size=224)),
                                      transforms.Lambda(lambda x: np.stack([x, x, x], axis=2)),
                                      transforms.ToPILImage('RGB'),
                                      transforms.RandomRotation(30),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([transforms.Lambda(lambda x: crop_resize(x, size=224)),
                                      transforms.Lambda(lambda x: np.stack([x, x, x], axis=2)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])


#print(features.shape)
#im = train_transform(features)
#print(im.shape)

In [7]:
class ClsTrainDataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df
        self.labels = self.df.iloc[:,-4:-1].values
        self.data = self.df.iloc[:, :-4].values
        self.torchtransforms = transforms

    def __getitem__(self, idx):
        HEIGHT = 137
        WIDTH = 236
        #row = self.df.iloc[idx].values
        #path = self.pathes[idx]
        img = self.data[idx, :]
        #print(img.shape)
        img = 255 - img.reshape(HEIGHT, WIDTH).astype(np.uint8)
        #print(img.shape)
        # img = crop_resize(img, size=224)
        # print(img.shape)
        #img = crop_resize(img)
        #img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)       
        #print(img.shape)
        # img = transforms.ToPILImage()(img)
        # print(img.)
        img = self.torchtransforms(img)
        
        label = self.labels[idx,:]
        return img, label
    def __len__(self):
        return len(self.df)

def make_loader(
        data_folder,
        batch_size=64,
        num_workers=2,
        is_shuffle = False,
):

    image_dataset = ClsTrainDataset(df = data_folder,
                                    transforms = train_transform)

    return DataLoader(
    image_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=True,
    shuffle=is_shuffle
    )

In [8]:
train_df_ = pd.read_csv("train.csv")

In [9]:
train_df = pd.merge(pd.read_parquet("train_image_data_0.parquet"), train_df_, on='image_id').drop(['image_id'], axis=1)

In [ ]:
validation_df = pd.merge(pd.read_parquet("train_image_data_1.parquet"), train_df_, on='image_id').drop(['image_id'], axis=1)

In [10]:
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss()
criterion3 = nn.CrossEntropyLoss()
epochs = 3
steps = 0
# running_loss = 0
# running_loss1 = 0
# running_loss2 = 0
# running_loss3 = 0
print_every = 5
# i = 0
# train_df = pd.merge(pd.read_parquet(f'train_image_data_{i}.parquet'), train_df_, on='image_id').drop(['image_id'], axis=1)

In [11]:
model_name = 'densenet121' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
model.last_linear = nn.Linear(model.last_linear.in_features, 186)

In [12]:
for param in model.parameters():
    param.requires_grad = True

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [14]:
optimizer = optim.Adam(model.parameters(), lr=0.003)
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [15]:
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss()
criterion3 = nn.CrossEntropyLoss()
epochs = 3
steps = 0

In [16]:
for epoch in range(epochs):
    dataloaders = make_loader(data_folder = train_df,
                                            batch_size=8,
                                            num_workers = 2,
                                            is_shuffle = False)
    accuracy1 = 0
    accuracy2 = 0
    accuracy3 = 0
    running_loss = 0
    running_loss1 = 0
    running_loss2 = 0
    running_loss3 = 0
    for images, labels in tqdm(dataloaders):
        steps += 1
        # Move input and label tensors to the default device
        images, labels = images.to(device), labels.to(device)
        label1 = labels[:,0]
        label2 = labels[:,1]
        label3 = labels[:,2]

        optimizer.zero_grad()
        
        ps = model.forward(images)
        logit1, logit2, logit3 = ps[:,: 168],\
                                    ps[:,168: 168+11],\
                                    ps[:,168+11:]
        top_p1, top_class1 = logit1.topk(1, dim=1)
        equals = top_class1 == label1.view(*top_class1.shape)
        accuracy1 += torch.mean(equals.type(torch.FloatTensor))
        top_p2, top_class2 = logit2.topk(1, dim=1)
        equals = top_class2 == label2.view(*top_class2.shape)
        accuracy2 += torch.mean(equals.type(torch.FloatTensor))
        top_p3, top_class3 = logit3.topk(1, dim=1)
        equals = top_class3 == label3.view(*top_class3.shape)
        accuracy3 += torch.mean(equals.type(torch.FloatTensor))
        loss1, loss2, loss3 = criterion1(logit1, label1),criterion2(logit2, label2),criterion3(logit3, label3)
        
        #print(loss1)
        (0.5*loss1 + 0.25*loss2 + 0.25*loss3).backward()
        
        optimizer.step()

        running_loss1 += loss1.item()
        running_loss2 += loss2.item()
        running_loss3 += loss2.item()
        running_loss = 0.5*running_loss1 + 0.25*running_loss2 + 0.25*running_loss3
    else:
        ## TODO: Implement the validation pass and print out the validation accuracy
        test_loss = 0
        accuracy = 0
        
        with torch.no_grad():
            for images, labels in testloader:
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
    print("Epoch: {}/{}.. ".format(epoch+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(dataloaders)),
              "Root Accuracy: {:.3f}".format(accuracy1/len(dataloaders)),
              "Vowel Accuracy: {:.3f}".format(accuracy2/len(dataloaders)),
              "Consonant Accuracy: {:.3f}".format(accuracy3/len(dataloaders)))
    

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]



Epoch: 1/3..  Training Loss: 2.577..  Train1 Accuracy: 0.123 Train2 Accuracy: 0.574 Train3 Accuracy: 0.701



Epoch: 2/3..  Training Loss: 1.074..  Train1 Accuracy: 0.558 Train2 Accuracy: 0.827 Train3 Accuracy: 0.864


OSError: [Errno 12] Cannot allocate memory

### Added 80/20 Split